# Meta Programming

In [ ]:
import time
from functools import wraps

def timeOfExecution(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        t_s = time.time()
        result = func(*args, **kwargs)
        t_e = time.time()
        print(func.__name__, t_e - t_s)
        return result
    return wrapper

# # # # # # # # "a" # # # # # # # #
@timeOfExecution
def counting(n):
    while n < 500000:
        n += 1
        
#### Main ####
counting(5)   

# # # # # # # # "b" # # # # # # # #
def counting(n : float):
    while n < 500000:
        n += 1
counting = timeOfExecution(counting)

#### Main ####
counting(5)      

# Annotations ::

In [1]:
def div(a, b):
    """Divide a by b args: a - the dividend b - the divisor (must be different than 0)
    return: the result of dividing a by b """
    return a / b


def div(a: 'the dividend', b: 'the divisor (must be different than 0)') -> 'the result of dividing a by b':
    """Divide a by b"""
    return a / b

def div(a: dict(type=float, help='the dividend'), b: dict(type=float, help='the divisor (must be different than 0)') )\
                -> dict(type=float, help='the result of dividing a by b') :
    """Divide a by b"""
    return a / b

print(div.__annotations__)

{'a': {'type': <class 'float'>, 'help': 'the dividend'}, 'b': {'type': <class 'float'>, 'help': 'the divisor (must be different than 0)'}, 'return': {'type': <class 'float'>, 'help': 'the result of dividing a by b'}}


In [ ]:
from inspect import signature
print(signature(counting)) # (n: float)

In [ ]:
from functools import wraps

def firstDecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('First Decorator')
        return func(*args, **kwargs)
    return wrapper

def secondDecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Second Decorator')
        return func(*args, **kwargs)
    return wrapper

@firstDecorator
@secondDecorator
def add(x, y):
    return x + y

#### Main ####
add(2,3)
add.__wrapped__(2,3)
add.__wrapped__.__wrapped__(2,3)

Decorator function that get arguments :

In [2]:
def decorator(arg1, arg2):
    def sub_decorator(func):
        def wrapper(*args, **kwargs):
            print('decorating with {} , {}'.format(arg1, arg2))
            return func(*args, **kwargs)
        return wrapper
    return sub_decorator

@decorator('arg1', 'arg2')
def abcd(*args):
    for arg in args:
        print(arg)
        
#### Main ####
abcd('a', 'b')

decorating with arg1 , arg2
a
b


In [4]:
from functools import wraps
import logging

def logged(level, name = None, message = None):
    def decorate(func):
        log_name = name if name else func.__module__
        log = logging.getLogger(log_name)
        log_msg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, log_msg)
            return func(*args, **kwargs)
        return wrapper

    return decorate

# # # # # # # # "a" # # # # # # # #
@logged(logging.DEBUG, 'NaMe', 'MSG')
def add(x, y, z):
    print(x + y + z)

#### Main ####
logging.basicConfig(level = logging.DEBUG)
add(1,2,3)

# # # # # # # # "b" # # # # # # # #
@logged(logging.WARNING)
def add(x, y, z):
    print(x + y + z)
    
#### Main ####
logging.basicConfig(level = logging.DEBUG)
add(1,2,3)

DEBUG:NaMe:MSG


6
6


Definition of decorator with user-configurable features:

In [ ]:
class MyClass():

    def __init__(self):
        self.__my_val = 10

    def set__my_val(self, val):
        self.__my_val = val

    def get__my_val(self):
        return self.__my_val


In [ ]:
x = 0
def func_out():
    x = 1
    def func_in():
        #nonlocal x
        #global x
        x = 2
        print('in: ', x)

    func_in()
    print('out: ', x)
#### Main ####
func_out()
print('global: ', x)

In [ ]:
from functools import partial

def func(x,y,z):
    return x*4 + 2*y + 5*z

new_add = partial(func, 2, 5)
print(new_add(3))

In [ ]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func = None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        log_name = name if name else func.__module__
        log = logging.getLogger(log_name)
        log_msg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, log_msg)
            return func(*args, **kwargs)

        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal log_msg
            log_msg = newmsg

        return wrapper
return decorate

@logged(logging.WARNING)
def add(x, y, z):
    return x + y + z

#### Main ####
logging.basicConfig(level = logging.DEBUG)
add.set_level(logging.DEBUG)
add.set_message("Hello")
add(1,2,3)


Definition of decorator as part of a class:

In [ ]:
from functools import wraps

class A:
    def firstDecorator(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('First decorator')
            return func(*args, **kwargs)
        return wrapper

    def secondDecorator(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Second decorator')
            return func(*args, **kwargs)
        return wrapper

    
a = A()

@a.firstDecorator
def abcd():
    pass

@a.secondDecorator
def dcba():
    pass

#### Main ####
abcd()
dcba()